In [2]:
%%capture
!pip install cyrtranslit
# !pip install transformers==4.28.0

In [3]:
import transformers
transformers.__version__

'4.30.1'

In [4]:
import my, data, net
from my import p

import sys
import numpy as np
import pandas as pd

import os
import gc
import matplotlib.pyplot as plt
import importlib
import pickle
import logging
from tqdm.notebook import tqdm

pd.set_option('display.max_rows', 200)
pd.set_option("max_colwidth", 85)
pd.set_option("display.precision", 1)
pd.options.display.float_format = "{:.5f}".format
# pd.set_option("display.max_rows", 5)
# pd.reset_option("display.max_rows")

from sklearn.model_selection import train_test_split

# from pandarallel import pandarallel
# pandarallel.initialize(progress_bar=True)
     
out = "out/"
da = '/kaggle/input/al-ner3/data/'

SEED = 34
np.random.seed(SEED)
N_CPU = os.cpu_count()
os.makedirs(out,exist_ok=True)

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [5]:
CHECKPOINT_DIR = out
os.makedirs(CHECKPOINT_DIR, exist_ok=True)
CHECKPOINT_DIR

'out/'

In [8]:
df = pd.read_csv(da + 'train_supervised_dataset_3.csv').fillna('')
df['good'] = df['good'].replace('товара нет', '')
p(df.shape)
df[:3]

(38000, 4)


,id,name,good,brand
0,0,Petmax Бантик леопард с красн розой 2шт,бантик,petmax
1,1,87191 Бусы для елки шарики_87191,бусы,
2,2,Футболка Piazza Italia WR011446881,футболка,piazza italia


In [9]:
df = data.correct_train(df)

numbers_in_bs = data.get_num(df.brand)
train_brands = set(df.loc[df['brand']!='','brand'].str.split(',').sum())

ru_words_in_targets = data.get_sep_ru_word(df.loc[df['good']!='','good'].to_list() +df.loc[df['brand']!='','brand'].to_list())
long_goods = data.get_long_goods(df.good)

chs_goods = data.get_replaces_goods(long_goods)
chs_brands = data.get_replaces_brands(train_brands, exclude=ru_words_in_targets)

def prepro(df, numbers_in_bs=None, chs_goods=None,chs_brands=None):
    df = df.copy()

    df['name'] = df['name'].apply(data.split_long_words)

    df['name'] = df['name'].apply(data.replace_to_ru)

    df['name'] = df['name'].apply(data.replace_misc)

    df['name'] = df['name'].apply(data.replace_num_0, args=(numbers_in_bs,))

    df['name'] = df['name'].apply(data.caps_to_low)

    df['text'] = df['name'].apply(data.del_space)

    df['name'] = df['name'].apply(data.correct_sent, args=(chs_goods,))
    df['name'] = df['name'].apply(data.correct_sent, args=(chs_brands,'title'))
    
    df['name'] = df['name'].apply(data.del_space)

    # df['text'] = df['text'].apply(data.del_space).str.lower()

    return df

df = prepro(df, numbers_in_bs, chs_goods, chs_brands)
df[:2]

,id,name,good,brand,text
0,0,Petmax Бантик Leopard с красн розой 0шт,бантик,petmax,Petmax Бантик леопард с красн розой 0шт
1,1,Бусы для елки шарики,бусы,,Бусы для елки шарики


In [10]:
df['good_1'] = df['good'].apply(lambda x:x.split(',')[0].strip())
df['brand_1'] = df['brand'].apply(lambda x:x.split(',')[0].strip())

ng, nb, ne_g, ne_b = data.stat_err(df)
ng, nb

(507, 1087)

In [11]:
from sklearn.preprocessing import LabelEncoder
from transformers import AutoTokenizer

encoders = {'good': LabelEncoder(), 'brand': LabelEncoder()}

tokenizer = AutoTokenizer.from_pretrained("cointegrated/LaBSE-en-ru")
len_text = my.len_in_tokens(df['name'], tokenizer, plot=False)

stratify = encoders['good'].fit_transform(df['good']) * 1000000 + encoders['brand'].fit_transform(df['brand']) * 100 + len_text

df = my.add_folds(df, strat_col=stratify, n_folds=6, seed=SEED)
df[:3]

StratifiedKFold


/opt/conda/lib/python3.10/site-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=6.
  warnings.warn(


,id,name,good,brand,text,good_1,brand_1,fold
0,0,Petmax Бантик Leopard с красн розой 0шт,бантик,petmax,Petmax Бантик леопард с красн розой 0шт,бантик,petmax,0
1,1,Бусы для елки шарики,бусы,,Бусы для елки шарики,бусы,,1
2,2,Футболка Piazza Italia @o,футболка,piazza italia,Футболка Piazza Italia @o,футболка,piazza italia,2


In [14]:
X_test = df[df.fold==0].reset_index(drop=True).copy()
X_test.shape

(6334, 8)

In [13]:
import torch
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping, TQDMProgressBar, ModelCheckpoint
from pytorch_lightning.plugins.precision import MixedPrecisionPlugin

torch.set_float32_matmul_precision('medium')
os.environ["TOKENIZERS_PARALLELISM"] = "false"

pl.__version__, torch.__version__

('2.0.3', '2.0.0')

In [15]:
class CFG:
    p_aug = 0.4
    train_full = False
    epochs=15
    tokenizer_name = 'cointegrated/LaBSE-en-ru'
    model_name = 'cointegrated/LaBSE-en-ru'
    num_labels = len(data.tag_to_index) - 1 # minus PAD

    hidden_dropout_prob = 0.1

    lr = 2e-5
    scheduler='cosine'
    num_cycles=0.5
    num_warmup_steps=30

    batch_size = 16
    val_bs = 16

    max_length = 100
    gradient_checkpointing=False
    gradient_accumulation_steps=1
    max_grad_norm=50
    precision = "16-mixed"

def train_fold(Xy:pd.DataFrame, Xy_test:pd.DataFrame, fold:int=0):
    print('[TRAIN FOLD]:',fold)
    net.set_seed(SEED + 10*fold)
    NAME_CKPT = f'best_f{fold}'

    dm = data.NerDataModule(Xy, Xy_test, fold=fold, n_cpu=2, cfg=CFG)

    CFG.num_train_steps = int(dm.len_train/CFG.batch_size*CFG.epochs)

    model = net.NerModule(CFG, lr = CFG.lr)

    tq = TQDMProgressBar(refresh_rate=5)

    es = EarlyStopping('f1_val', min_delta=0.001,patience=50,verbose=True, mode='max', check_on_train_epoch_end=False)

    chpt = ModelCheckpoint(dirpath=CHECKPOINT_DIR,filename=f'best_f{fold}',  monitor='f1_val',mode='max')

    trainer = pl.Trainer(
        precision=CFG.precision,
        plugins=[MixedPrecisionPlugin(precision='16-mixed',device='cuda')],
    callbacks=[tq,es,chpt],
    max_epochs=CFG.epochs,
    deterministic = True,
    accelerator='auto',
    accumulate_grad_batches = CFG.gradient_accumulation_steps,
    gradient_clip_val = CFG.max_grad_norm,
    log_every_n_steps = 50,
    enable_model_summary = True if fold==0 else False,
    # val_check_interval = 1.,
#         auto_lr_find=True
    )
    trainer.fit(model, datamodule=dm)

#     del trainer
    torch.cuda.empty_cache()
    gc.collect()

    model = net.NerModule.load_from_checkpoint(checkpoint_path=chpt.best_model_path).to(net.device)

    preds = trainer.predict(model, dm)

    return preds

preds = train_fold(df, X_test, fold=0)

[TRAIN FOLD]: 0


Some weights of the model checkpoint at cointegrated/LaBSE-en-ru were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Sanity Checking: 0it [00:00, ?it/s]

> VAL: f1: 0.045 | f1_G: 0.066 | f1_B: 0.034 | PR_G: 0.045 | RE_G: 0.125 | PR_B: 0.023 | RE_B: 0.062


Training: 0it [00:00, ?it/s]

...

> VAL: f1: 0.866 | f1_G: 0.925 | f1_B: 0.837 | PR_G: 0.922 | RE_G: 0.928 | PR_B: 0.835 | RE_B: 0.838
TRAIN: f1: 0.966 | f1_G: 0.977 | f1_B: 0.960 | PR_G: 0.976 | RE_G: 0.977 | PR_B: 0.960 | RE_B: 0.961


Validation: 0it [00:00, ?it/s]

> VAL: f1: 0.866 | f1_G: 0.925 | f1_B: 0.836 | PR_G: 0.922 | RE_G: 0.927 | PR_B: 0.835 | RE_B: 0.838
TRAIN: f1: 0.966 | f1_G: 0.977 | f1_B: 0.961 | PR_G: 0.976 | RE_G: 0.977 | PR_B: 0.960 | RE_B: 0.961


Some weights of the model checkpoint at cointegrated/LaBSE-en-ru were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Predicting: 0it [00:00, ?it/s]

In [17]:
!rm -rf out/*

In [18]:
class CFG:
    p_aug = 0.4
    train_full = True
    epochs=15
    tokenizer_name = 'cointegrated/LaBSE-en-ru'
    model_name = 'cointegrated/LaBSE-en-ru'
    num_labels = len(data.tag_to_index) - 1 # minus PAD

    hidden_dropout_prob = 0.1

    lr = 2e-5
    scheduler='cosine'
    num_cycles=0.5
    num_warmup_steps=30

    batch_size = 16
    val_bs = 16

    max_length = 100
    gradient_checkpointing=False
    gradient_accumulation_steps=1
    max_grad_norm=50
    precision = "16-mixed"

def train_fold(Xy:pd.DataFrame, Xy_test:pd.DataFrame, fold:int=0):
    print('[TRAIN FOLD]:',fold)
    net.set_seed(SEED + 10*fold)
    NAME_CKPT = f'best_f{fold}'

    dm = data.NerDataModule(Xy, Xy_test, fold=fold, n_cpu=2, cfg=CFG)

    CFG.num_train_steps = int(dm.len_train/CFG.batch_size*CFG.epochs)

    model = net.NerModule(CFG, lr = CFG.lr)

    tq = TQDMProgressBar(refresh_rate=5)

    es = EarlyStopping('f1_val', min_delta=0.001,patience=50,verbose=True, mode='max', check_on_train_epoch_end=False)

    chpt = ModelCheckpoint(dirpath=CHECKPOINT_DIR,filename=f'best_f{fold}',  monitor='f1_val',mode='max')

    trainer = pl.Trainer(
        precision=CFG.precision,
        plugins=[MixedPrecisionPlugin(precision='16-mixed',device='cuda')],
    callbacks=[tq,es,chpt],
    max_epochs=CFG.epochs,
    deterministic = True,
    accelerator='auto',
    accumulate_grad_batches = CFG.gradient_accumulation_steps,
    gradient_clip_val = CFG.max_grad_norm,
    log_every_n_steps = 50,
    enable_model_summary = True if fold==0 else False,
    # val_check_interval = 1.,
#         auto_lr_find=True
    )
    trainer.fit(model, datamodule=dm)

#     del trainer
    torch.cuda.empty_cache()
    gc.collect()

    model = net.NerModule.load_from_checkpoint(checkpoint_path=chpt.best_model_path).to(net.device)

    preds = trainer.predict(model, dm)

    return preds

preds = train_fold(df, X_test, fold=0)

[TRAIN FOLD]: 0


Some weights of the model checkpoint at cointegrated/LaBSE-en-ru were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Sanity Checking: 0it [00:00, ?it/s]

> VAL: f1: 0.045 | f1_G: 0.066 | f1_B: 0.034 | PR_G: 0.045 | RE_G: 0.125 | PR_B: 0.023 | RE_B: 0.062


Training: 0it [00:00, ?it/s]

...

> VAL: f1: 0.962 | f1_G: 0.975 | f1_B: 0.955 | PR_G: 0.974 | RE_G: 0.977 | PR_B: 0.955 | RE_B: 0.955
TRAIN: f1: 0.968 | f1_G: 0.977 | f1_B: 0.963 | PR_G: 0.977 | RE_G: 0.978 | PR_B: 0.962 | RE_B: 0.964


Validation: 0it [00:00, ?it/s]

> VAL: f1: 0.962 | f1_G: 0.975 | f1_B: 0.955 | PR_G: 0.974 | RE_G: 0.977 | PR_B: 0.955 | RE_B: 0.955
TRAIN: f1: 0.967 | f1_G: 0.977 | f1_B: 0.962 | PR_G: 0.976 | RE_G: 0.977 | PR_B: 0.961 | RE_B: 0.962


Some weights of the model checkpoint at cointegrated/LaBSE-en-ru were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Predicting: 0it [00:00, ?it/s]

In [ ]:
def fn_good(x):
    if x:
        x = [el.strip().strip('-./') for el in x]
        return ','.join(x)
    else:
        return ''

def fn_brand(x):
    if x:
        x = [el.strip().strip('-./') for el in x]
        if x:
            return x[0]
    
    return ''
    
sub = pd.DataFrame(sum(preds, list()), columns=["good", "brand"])


sub['good'] = sub['good'].apply(fn_good)
sub['brand'] = sub['brand'].apply(fn_brand)

sub['true_good'] = X_test['good']
sub['true_brand'] = X_test['brand']
sub['name'] = X_test['name']
sub['id'] = X_test['id']
# sub = sub[["id", "good", "brand",'name']]
sub

In [ ]:
sub.to_csv("val_sub_fold_0.csv", index=False)

In [ ]:
import re

gs = []
for t in sub.itertuples():
    name = t.name.lower()
    new_goods = []
    if t.good == '':
        gs.append('')
        continue
    for g in t.good.split(','):
        n1 = name.find(g)
        if (n1 > 0) and re.findall('[А-я]',name[n1-1]):
            print(0, t, name[n1-1])
            continue
        if n1+len(g) < len(name) and re.findall('[А-я]',name[n1+len(g)]):
            print(1, t,name[n1+len(g)])
            continue
        new_goods.append(g)
    gs.append(','.join(new_goods))
        
len(gs)

In [ ]:
bs = []
for t in sub.itertuples():
    name = t.name.lower()
    new_brands = []
    b = t.brand
    if b == '':
        bs.append('')
        continue
    n1 = name.find(b)
    if (n1 > 0) and re.findall('[А-яA-Za-z]',name[n1-1]):
        print(0, t, name[n1-1])
        bs.append('')
        continue
    if n1+len(b) < len(name) and re.findall('[А-яA-Za-z]',name[n1+len(b)]):
        print(1, t,name[n1+len(b)])
        bs.append('')
        continue
    bs.append(b)
        
len(bs)

In [ ]:
sub['good'] = gs
sub['brand'] = bs
sub[:3]

In [ ]:
sub.drop(columns='name').to_csv('s10.csv',index=False)